### FigS5g

In [ ]:
library(DESeq2)
library(EnhancedVolcano)
library(dplyr)

In [ ]:
miRNA_data <- read.csv("./EC_OC_raw_miRNA.csv",row.names=1)
colnames(miRNA_data) <- sub("^.", "", colnames(miRNA_data))
colnames(miRNA_data) <- sub("\\..*$", "", colnames(miRNA_data))


In [ ]:
clinical_data <- read.csv("./bulkRNA_clini_info.csv")

clinical_data$Tumor_SubType <- factor(clinical_data$Tumor_SubType,levels=c("Mucinous","Serous","Clear cell","Adenocarcinoma","Adenosquamous","Squamous","Endometrioid","Sarcoma","Others"))
clinical_data$Tumor_Type <- factor(clinical_data$Tumor_Type,levels=c("OC","EC","CC"))
clinical_data <- clinical_data[order(clinical_data$cluster, clinical_data$Tumor_Type, clinical_data$Tumor_SubType), ]
selected_data <- clinical_data[(clinical_data$Tumor_SubType=="Serous") | (clinical_data$Tumor_SubType=="Endometrioid"),]


In [ ]:
mirna_names <- colnames(miRNA_data)
rna_names <- selected_data$sampleName
both_names <- intersect(mirna_names, rna_names)
selected_mirna_data <- selected_data[selected_data$sampleName %in% both_names,]
select_miRNA_data <- miRNA_data[,selected_mirna_data$sampleName]
select_miRNA_data <- as.matrix(select_miRNA_data)


In [ ]:
samples_1 <- data.frame(
    row.names = selected_mirna_data$sampleName,
    condition = ifelse(selected_mirna_data$Tumor_SubType=="Serous", "Serous", "Endometrioid")
)
samples_1$condition <- as.factor(samples_1$condition)
dds_1 <- DESeqDataSetFromMatrix(countData = select_miRNA_data, colData = samples_1, design = ~ condition)
dds_1 <- DESeq(dds_1)

res_1 <- results(dds_1, contrast=c("condition", "Serous", "Endometrioid"))
res_1 <- res_1[order(res_1$padj),]
df_1 <- data.frame(res_1)
filtering_res_1 <- subset(res_1, padj < 0.001 & abs(log2FoldChange) > 1.5)
filtering_res_1 <- subset(filtering_res_1, baseMean >20)
filtering_df_1 <- data.frame(filtering_res_1)


In [ ]:
vol_1 <- EnhancedVolcano(res_1,
    lab = rownames(res_1),
    x = 'log2FoldChange',
    y = 'padj',
    selectLab = row.names(filtering_df_1),
    pCutoff = 0.001,
    FCcutoff = 1.5,
    ylim = c(0, 10),
    xlim = c(-5, 6),
    # drawConnectors = TRUE,
    # colConnectors = 'gray50',
    pointSize = 3.0,
    labSize = 6.0)
